_Run Cell 1 without change_ (Included in all my notebooks because I use jupyter lab templates.)

In [13]:
import sys
import time
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
import numpy as np
import pandas as pd
from pathlib import Path

# To get multiple outputs into 1 cell w/o using print:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# autoreload extension
from IPython import get_ipython
ipython = get_ipython()
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

# -----------------------------------------
# USEFUL FUNCTIONS:

def add_to_sys_path(this_path, up=False):
    """
    To be used if the current project is not (yet)  packaged.
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    if up:
        newp = Path(this_path).parent
    else:
        newp = Path(this_path)
    src = newp.joinpath("src")
    if src.exists():
        newp = str(src)
    else:
        newp = str(newp)
    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))


# Filtered dir() for method discovery:
def fdir(obj, start_with_str='_', exclude=True):
    return [d for d in dir(obj) if not d.startswith(start_with_str) == exclude]


def show_elapsed_time(start_t:time):
    elapsed = time.time() - start_t
    print(f"Elapsed time: {elapsed:,.2f} s ({elapsed/60:,.2f} min).")
    return

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Insert current src dir into sys.path so that modules in ../src can be imported:
# CHANGE THIS IF NEEDED (up=True if this notebook is inside a folder):

add_to_sys_path(Path.cwd(), up=True)

Path added to sys.path: /home/cat/projects/Microstate-Code


# The `ms_analysis module` main functions:
 * read the microstate file
 * convert the microstates to charge microstates

Note: the ms_analysis.py module is now the most up-to-date one.

In [4]:
from analyze import ms_analysis as msa

---
#### Optional inspection: if you want to know the methods/variables in an object:

In [5]:
fdir(msa)

['Charge_Microstate',
 'Conformer',
 'Kcal2kT',
 'MSout',
 'Microstate',
 'Path',
 'Union',
 'bhata_distance',
 'conformers',
 'e2occ',
 'fixed_residues_charge',
 'free_residues_df',
 'groupms_byconfid',
 'groupms_byenergy',
 'groupms_byiconf',
 'math',
 'ms_charge',
 'ms_convert2occ',
 'ms_convert2sumcrg',
 'ms_counts',
 'ms_energy_stat',
 'ms_to_charge_ms',
 'np',
 'operator',
 'pd',
 'ph2Kcal',
 'read_conformers',
 'sort_charge_microstates',
 'sys',
 'topN_charge_microstates',
 'whatchanged_conf',
 'whatchanged_res',
 'zlib']

In [6]:
fdir(msa.MSout)

['get_sampled_ms', 'load_msout', 'sort_microstates']

In [7]:
fdir(msa.Charge_Microstate)

['state']

## PRE-REQUISITES:
You can access a MCCE output folder that contains a `ms_out` subfolder, which contains as many .txt files as titration points
whose names follow this pattern: pH<X>eH<Y>ms.txt.  
Thus, the file `pH7eH0ms.txt` records the microstates information at pH 7 and eH  0.

### Note:
You do not have to run this notebook inside an MCCE output folder.

### Checks on folders and files:

In [9]:
# Valid mcce dir (for Cat):

MCCE_DIR = Path.cwd().joinpath("mcce_data")
MCCE_MSOUT = MCCE_DIR.joinpath("ms_out")
if not MCCE_MSOUT.exists():
    print(f"{MCCE_MSOUT.name!r} not found in {MCCE_MSOUT.parent!r}, but required.\n",
         "The folder is retained when step4.py is run with the '--ms' option.")

## Load conformers

#### New behavior:
Since you provide a path to mcce data &mdash; via MCCE_DIR &mdash; instead of moving python modules to the data, 
the `msa.conformers` list will be empty when ms_analysis.py is imported outside an MCCE output folder.  
**The following cell performs that check:**

In [10]:
conformers = msa.conformers  # : will be [] when run outside an MCCE output folder
if len(conformers) == 0:
    conformers = msa.read_conformers(MCCE_DIR.joinpath("head3.lst"))

N_conformers = len(conformers)
print(f"{N_conformers = :,}")

N_conformers = 1,673


### Define often used dictionnaries from conformers:

In [11]:
conf_crg_from_id = dict((conf.iconf, conf.crg) for conf in conformers)

## Read the microstate file for a given pH:

In [12]:
msout_file = MCCE_MSOUT.joinpath("pH7eH0ms.txt")
if not msout_file.exists():
    print(f"{msout_file.name} not found!")

In [14]:
start_t = time.time()

mc = msa.MSout(msout_file)

show_elapsed_time(start_t)

print(f"Microstates:\n\tTotal count: {mc.N_ms = :,}\n\tUnique microstates: {mc.N_uniq = :,}")

Elapsed time: 105.37 s (1.76 min).
Microstates:
	Total count: mc.N_ms = 9,000,000
	Unique microstates: mc.N_uniq = 3,086,617


In [20]:
N_free_res = len(mc.free_residues)
print(f"{N_free_res = :,}")

# List of list: [conformer indices for each free residue]
mc.free_residues[:5]

N_free_res = 145


[[0, 1, 3, 4, 5, 6, 14, 16], [20, 21], [24, 25], [31, 32, 35, 36], [38, 39]]

## Change mc.microstates to a list

In [16]:
print(f"{type(mc.microstates) = }")

if isinstance(mc.microstates, dict):
    microstates = list(mc.microstates.values())
else:
    microstates = mc.microstates

microstates[:4]

type(mc.microstates) = <class 'dict'>


---
## For 'MCCE to MD' project: Convert microstates to charge_microstates

In [17]:
charge_ms = msa.ms_to_charge_ms(microstates, conformers)
N_charge_ms = len(charge_ms)
print(f"{N_charge_ms = :,}")

N_charge_ms = 768


### Get the top1 charge microstate by count (i.e. the most frequent)

In [24]:
top1_cms_by_count = msa.topN_charge_microstates(charge_ms, sort_by="count")  # N=1, sort_reverse=True :: defaults
print("top1_cms_by_count[0]:\n", top1_cms_by_count[0])

top1_by_count_state_size = len(top1_cms_by_count[0].state())
print(f"{top1_by_count_state_size = :,};\nCheck: same as N_free_res? {N_free_res==top1_by_count_state_size}\n")

top1_cms_by_count[0]:
 Charge_Microstate(
	count = 1,232,506,
	E = 4,692.84,
	state = [0, 0, 0, -1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0, 1, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0]
)
top1_by_count_state_size = 145;
Check: same as N_free_res? True



### Get the top1 charge microstate by energy (i.e. the most favorable)

In [25]:
top1_cms_by_energy = msa.topN_charge_microstates(charge_ms, sort_by="E", sort_reverse=False)  # N=1 :: default
print(top1_cms_by_energy[0])

top1_by_E_state_size = len(top1_cms_by_energy[0].state())
print(f"{top1_by_E_state_size = :,}; Check: same as N_free_res? {N_free_res==top1_by_E_state_size}")

Charge_Microstate(
	count = 81,
	E = 4,688.93,
	state = [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, 0, -1, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0, 1, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0]
)
top1_by_E_state_size = 145; Check: same as N_free_res? True


## In MCCE, residues are grouped into free and fixed residues.
 * Free residues appear in the microstate id list but fixed residues do not.
 * Fixed residues can have the titrating residues so it is important to look up the fixed residues and their contributing total charge is called here as background charge.

## Free residues information

In [26]:
free_res_df = msa.free_residues_df(mc.free_residues, conformers)
free_res_df

,FreeRes
0,META0001_
1,TYRA0004_
2,ASNA0006_
3,GLUA0008_
4,THRA0010_
...,...
140,HOHZ0010_
141,HOHZ0068_
142,HOHZ0069_
143,HOHZ0070_


## Fixed residues charge
Make sure to get the fixed residues charge for MD protonation input.

In [27]:
background_charge, fixed_res_crg_dict = msa.fixed_residues_charge(conformers, mc.fixed_iconfs)
print(f"{background_charge = :.2f}")

background_charge = -6.00


## This will keep only fixed residues of interest and their charge information. 


### Note: Make sure to add your all interested residues name here (such as quinone)

In [28]:
res_of_interest = ["ASP", "MQ8", "GLU", "ARG", "HIS", "LYS", "SER", "CYS", "THR", "TYR", "NTR", "CTR"]
len(res_of_interest)

12

### Keep only your interested residue information in fixed residues

#### Note:
In Chun's app, the "crg" column name will be an index indentifying each vector requested, 
e.g.: "top 5" -> "0 1 2 3 4" or "1 2 3 4 5".

In [29]:
fixed_res_of_interest = {i:j for i,j in fixed_res_crg_dict.items() if i[:3] in res_of_interest}
df_fixed_res_crg = pd.DataFrame(fixed_res_of_interest.items(), columns=['FixedRes', 'crg'])
df_fixed_res_crg.shape[0]
df_fixed_res_crg

24

,FixedRes,crg
0,THRA0003_,-0.0
1,ASPA0012_,-1.0
2,THRA0054_,-0.0
3,THRA0084_,-0.0
4,THRA0086_,-0.0
5,TYRA0087_,0.0
6,THRA0111_,-0.0
7,ASPA0117_,-1.0
8,ASPA0131_,-1.0
9,ASPA0134_,-1.0


## Next:
  * combine Free & Fixed data for each "top N" request.
  * filter out default charges
  * save to csv